## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import pywt

## 데이터 크기 정의하기

In [ ]:
NoOfData    = 180  # 정상/고장 데이터 각 180개씩 
NoOfSensor  = 3    # 가속도(Acceleration), 전압(Voltage), 전류(Current)
NoOfFeature = 10   # 특징 개수:10개 (순서: Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Impulse factor, Shape factor)

NoOfData, NoOfSensor, NoOfFeature

## Time Domain 특징값 추출 (10 features * 3 sensors = 30개씩)

In [ ]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [ ]:
# 특징데이터 크기 지정
TimeFeature_Normal   = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
TimeFeature_Abnormal = np.zeros((NoOfSensor*NoOfFeature , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_data2 = pd.read_csv(temp_path2 , sep=',' , header=None) # 임시 Abnormal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[10*j+0, i] = np.max(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+1, i] = np.min(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+2, i] = np.mean(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+3, i] = rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+4, i] = np.var(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+5, i] = sp.skew(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+7, i] = np.max(temp_data1.iloc[:,j+1])/rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+8, i] = rms(temp_data1.iloc[:,j+1])/np.mean(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+9, i] = np.max(temp_data1.iloc[:,j+1])/np.mean(temp_data1.iloc[:,j+1])
        
        # Abnormal Time Domain Feature
        TimeFeature_Abnormal[10*j+0, i] = np.max(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+1, i] = np.min(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+2, i] = np.mean(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+3, i] = rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+4, i] = np.var(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+5, i] = sp.skew(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+6, i] = sp.kurtosis(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+7, i] = np.max(temp_data2.iloc[:,j+1])/rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+8, i] = rms(temp_data2.iloc[:,j+1])/np.mean(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+9, i] = np.max(temp_data2.iloc[:,j+1])/np.mean(temp_data2.iloc[:,j+1])
        
print(TimeFeature_Normal.shape)
print(TimeFeature_Abnormal.shape)

Normal, Abnormal 데이터 합치기  

In [ ]:
TimeFeature = np.concatenate([TimeFeature_Normal, TimeFeature_Abnormal] , axis=1)
TimeFeature.shape

## Frequncy Domain 특징값 추출 (10 features \* 8 wavelet levels * 3 sensors = 240개씩)

In [ ]:
# Wavelet options

MotherWavelet = pywt.Wavelet('db4')   # Mother wavelet (모함수) 지정
Level   = 8                    # Wavelet 분해 레벨 지정
select  = 8                    # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [ ]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))
FreqFeature_Abnormal = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './SpotWeldingData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './SpotWeldingData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = np.array(pd.read_csv(temp_path1 , sep=',', header=None).iloc[:,1:]) # 임시 Normal 데이터
    temp_data2 = np.array(pd.read_csv(temp_path2 , sep=',', header=None).iloc[:,1:]) # 임시 Abnormal 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    Coef2      = pywt.wavedec(temp_data2, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            coef2 = Coef2[Level-k]
            
            
            # Normal Frequency Domain Feature
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , (i-1)] = np.max(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , (i-1)] = np.min(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , (i-1)] = np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , (i-1)] = np.var(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , (i-1)] = rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , (i-1)] = sp.skew(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , (i-1)] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , (i-1)] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , (i-1)] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , (i-1)] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
            # Abnormal Frequency Domain Feature
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+0 , (i-1)] = np.max(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+1 , (i-1)] = np.min(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+2 , (i-1)] = np.mean(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+3 , (i-1)] = np.var(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+4 , (i-1)] = rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+5 , (i-1)] = sp.skew(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+6 , (i-1)] = sp.kurtosis(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+7 , (i-1)] = np.max(coef2[:,j])/rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+8 , (i-1)] = rms(coef2[:,j])/np.mean(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+9 , (i-1)] = np.max(coef2[:,j])/np.mean(coef2[:,j])

print(FreqFeature_Normal.shape)
print(FreqFeature_Abnormal.shape)

Normal, Abnormal 특징값 합치기      

In [ ]:
FreqFeature = np.concatenate([FreqFeature_Normal, FreqFeature_Abnormal] , axis=1)
FreqFeature.shape

## Feature 통합 (Time domain 30개 + Freq. domain 240 = 270개)

In [ ]:
Features = np.concatenate([TimeFeature,FreqFeature] , axis=0)

print("Feature Data Size :", Features.shape)
print("= 데이터 360개(정상/고장 각 180개씩)가 각각 270개의 특징값으로 구성됨")

저장에 용이한 Data frame 형식으로 변환

In [ ]:
FeatureData = pd.DataFrame(Features)
FeatureData

추출된 특징 데이터 저장 (.csv 파일)

In [ ]:
path = './ProcessdData/FeatureData'   # path = '파일 경로/저장할 파일 이름'
FeatureData.to_csv(path, sep=',', header=None , index=None)

# [실습 과제 2]

## 1. 센서 데이터 2개 (전압, 전류) 및 WT level 6으로 설정하여 다른 형태의 특징데이터 추출 
#### >>>>>> 저장된 특징 데이터 파일 제출
#### >>>>>> 데이터 이름 : ST(수강생 번호)_HW2_1  (예시 : 'ST000_HW2_1'  //  'ST00_HW2_1'  //  'ST0_HW2_1')
#### >>>>>> 데이터 이름 중 'ST' , 'HW' 등 영어는 모두 대문자

### ****** 필독 !! 실습과제 주의사항 ******

- 각자의 "수강생 번호" 확인 (아이캠퍼스 공지) 
- 제출하는 실습과제 파일에 "수강생 번호"를 기준으로 작성 (이름, 학번 등 작성X)
- 각 실습과제에 대한 구체적인 파일 이름은 매번 개별 안내 
  (수강생 번호 123번 학생 과제파일 예시 : 'ST123_HW2_1.csv' )
- 과제 파일이름 양식 지키지 않을 시 감점!